## Compulsory Task 

In this compulsory task you will clean the country column and parse the date column in the **store_income_data_task.csv** file.

In [2]:
import pandas as pd
import chardet as cd
# Load up store_income_data.csv
income_df = pd.read_csv('store_income_data_task.csv')

# Display the first few rows to inspect the data
income_df.head()


,id,store_name,store_email,department,income,date_measured,country
0,1,"Cullen/Frost Bankers, Inc.",NaN,Clothing,$54438554.24,4-2-2006,United States/
1,2,Nordson Corporation,NaN,Tools,$41744177.01,4-1-2006,Britain
2,3,"Stag Industrial, Inc.",NaN,Beauty,$36152340.34,12-9-2003,United States
3,4,FIRST REPUBLIC BANK,ecanadine3@fc2.com,Automotive,$8928350.04,8-5-2006,Britain/
4,5,Mercantile Bank Corporation,NaN,Baby,$33552742.32,21-1-1973,United Kingdom


1. Take a look at all the unique values in the "country" column. Then, convert the column to lowercase and remove any trailing white spaces.

In [3]:
# Check the unique values before cleaning
unique_before = income_df['country'].unique()
print("Unique country values before cleaning:", unique_before)
print(f"\nThere are {len(unique_before)} unique values before cleaning.")

# Convert to lowercase
income_df['country'] = income_df['country'].str.lower()

# Remove white spaces
income_df['country'] = income_df['country'].str.strip()

# Remove trailing/leading spaces
income_df['country'] = income_df['country'].str.rstrip('/.')
income_df['country'] = income_df['country'].replace('', pd.NA)
income_df.dropna(subset=['country'], inplace=True)

# Check unique values after cleaning basic formatting
unique_after = income_df['country'].unique()
print("\nUnique country values after basic cleaning:", unique_after)
print(f"\nThere are {len(unique_after)} unique values after basic cleaning.")

Unique country values before cleaning: ['United States/' 'Britain' ' United States' 'Britain/' ' United Kingdom'
 'U.K.' 'SA ' 'U.K/' 'America' 'United Kingdom' nan 'united states'
 ' S.A.' 'England ' 'UK' 'S.A./' 'ENGLAND' 'BRITAIN' 'U.K' 'U.K '
 'America/' 'SA.' 'S.A. ' 'u.k' 'uk' ' ' 'UK.' 'England/' 'england'
 ' Britain' 'united states of america' 'UK/' 'SA/' 'SA' 'England.'
 'UNITED KINGDOM' 'America.' 'S.A..' 's.a.' ' U.K'
 ' United States of America' 'Britain ' 'England' ' SA'
 'United States of America.' 'United States of America/' 'United States.'
 's. africasouth africa' ' England' 'United Kingdom '
 'United States of America ' ' UK' 'united kingdom' 'AMERICA' 'America '
 'UNITED STATES OF AMERICA' ' S. AfricaSouth Africa' 'america'
 'S. AFRICASOUTH AFRICA' 'Britain.' '/' 'United Kingdom.' 'United States'
 ' America' 'UNITED STATES' 'sa' 'United States of America' 'UK '
 'United States ' 'S. AfricaSouth Africa/' 'S.A.' 'United Kingdom/'
 'S. AfricaSouth Africa ' 'S. AfricaSou

2. Note that there should only be three separate countries. Eliminate all variations, so that 'South Africa', 'United Kingdom' and 'United States' are the only three countries.

In [4]:
import fuzzywuzzy
from fuzzywuzzy import process, fuzz


def replace_matches_in_column(df, column, string_to_match, min_ratio = 60):
    """
    Replace entries in df[column] that closely match string_to_match with string_to_match.
    
    Parameters:
        df (DataFrame): The DataFrame containing the column to modify.
        column (str): The name of the column to process.
        string_to_match (str): The standardized string to replace close matches with.
        min_ratio (int): The minimum fuzzy match ratio to consider for replacement.
    """
    # Get the unique non-null strings from the column
    unique_strings = df[column].dropna().unique()
    
    # Use fuzzy matching to extract the closest matches to string_to_match
    matches = process.extract(string_to_match, unique_strings, scorer=fuzz.token_sort_ratio, limit=10)
    
    # Filter out matches that don't meet the minimum ratio
    close_matches = [match[0] for match in matches if match[1] >= min_ratio]
    
    # Replace all rows that have any of these close matches with the standardised string
    df.loc[df[column].isin(close_matches), column] = string_to_match
    
    print(f"Replaced the following matches with '{string_to_match}': {close_matches}")


replace_matches_in_column(income_df, 'country', 'united states')
replace_matches_in_column(income_df, 'country', 'united kingdom')
replace_matches_in_column(income_df, 'country', 'south africa')

# Verify the unique values after applying fuzzy matching
unique_final = sorted(income_df['country'].dropna().unique())
print("Unique country values after fuzzy matching:", unique_final)


Replaced the following matches with 'united states': ['united states', 'united states of america']
Replaced the following matches with 'united kingdom': ['united kingdom']
Replaced the following matches with 'south africa': ['s. africasouth africa']
Unique country values after fuzzy matching: ['america', 'britain', 'england', 's.a', 'sa', 'south africa', 'u.k', 'uk', 'united kingdom', 'united states']


c:\Users\jorot\AppData\Local\Programs\Python\Python313\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [5]:
# Define a mapping for known country variants
mapping = {
    # United States variants
    'america': 'united states',
    
    # United Kingdom variants
    'britain': 'united kingdom',
    'u.k': 'united kingdom',
    'uk': 'united kingdom',
    'england': 'united kingdom',
    
    # South Africa variants
    's.a': 'south africa',
    'sa': 'south africa'
}

# Apply the mapping
income_df['country'] = income_df['country'].replace(mapping)

# Verify the unique country values after mapping
unique_final = income_df['country'].dropna().unique()
print("Unique country values after mapping:", unique_final)

Unique country values after mapping: ['united states' 'united kingdom' 'south africa']


3. Create a new column called `days_ago` in the DataFrame that is a copy of the 'date_measured' column but instead it is a number that shows how many days ago it was measured from the current date. Note that the current date can be obtained using `datetime.date.today()`.

In [ ]:
import datetime as dt

# First, parse the 'date_measured' column to datetime.
income_df['date_parsed'] = pd.to_datetime(income_df['date_measured'], format='%d-%m-%Y')

# Get today's date as a Python date, then convert to pandas Timestamp and normalise it
today = pd.Timestamp(dt.date.today()).normalize()

# Compute the number of days ago
income_df['days_ago'] = (today - income_df['date_parsed']).dt.days

# Display the result with the new column added
print(income_df[['date_measured', 'date_parsed', 'days_ago']].head())

2025-03-06 00:00:00
  date_measured date_parsed  days_ago
0      4-2-2006  2006-02-04      6970
1      4-1-2006  2006-01-04      7001
2     12-9-2003  2003-09-12      7846
3      8-5-2006  2006-05-08      6877
4     21-1-1973  1973-01-21     19037
